<a href="https://colab.research.google.com/github/serdarbozoglan/My_Pyspark/blob/master/SparkNLP_Training02_TextProcessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Code is from GitHub

https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Public/2.Text_Preprocessing_with_SparkNLP_Annotators_Transformers.ipynb

In [1]:
import os

# Install java
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! java -version

# Install pyspark
! pip install --ignore-installed pyspark==2.4.4

# Install Spark NLP
! pip install --ignore-installed spark-nlp==2.4.5

openjdk version "1.8.0_242"
OpenJDK Runtime Environment (build 1.8.0_242-8u242-b08-0ubuntu3~18.04-b08)
OpenJDK 64-Bit Server VM (build 25.242-b08, mixed mode)
     |████████████████████████████████| 215.7MB 70kB/s 
     |████████████████████████████████| 204kB 53.6MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216130388 sha256=b6f69b742a13c9b22c318bf646d7501273d575705c0ea86e57bca30512657f5c
  Stored in directory: /root/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471
Successfully built pyspark
     |████████████████████████████████| 112kB 13.1MB/s 


In [0]:
import pandas as pd

## Start Spark

In [4]:
import sparknlp
spark = sparknlp.start()

print('Spark NLP Version : ', sparknlp.version())
print('Apache Spark Version :', spark.version)

Spark NLP Version :  2.4.5
Apache Spark Version : 2.4.4


In [6]:
!wget https://gist.githubusercontent.com/vkocaman/e091605f012ffc1efc0fcda170919602/raw/fae33d25bd026375b2aaf1194b68b9da559c4ac4/annotators.csv

--2020-04-20 13:06:50--  https://gist.githubusercontent.com/vkocaman/e091605f012ffc1efc0fcda170919602/raw/fae33d25bd026375b2aaf1194b68b9da559c4ac4/annotators.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3136 (3.1K) [text/plain]
Saving to: ‘annotators.csv’

annotators.csv      100%[===================>]   3.06K  --.-KB/s    in 0s      

2020-04-20 13:06:50 (49.3 MB/s) - ‘annotators.csv’ saved [3136/3136]



In [0]:
import pandas as pd

df = pd.read_csv('annotators.csv')

In [9]:
df

,Annotator,Description,Version,Annotator Approach,Annotator Model
0,Tokenizer*,Identifies tokens with tokenization open stand...,Opensource,-,+
1,Normalizer*,Removes all dirty characters from text,Opensource,-,+
2,Stemmer*,Returns hard'-stems out of words with the obje...,Opensource,+,-
3,Lemmatizer*,Retrieves lemmas out of words with the objecti...,Opensource,-,+
4,RegexMatcher*,Uses a reference file to match a set of regula...,Opensource,+,+
5,TextMatcher*,Annotator to match entire phrases (by token) p...,Opensource,+,+
6,Chunker*,Matches a pattern of part'-of'-speech tags in ...,Opensource,+,-
7,DateMatcher*,Reads from different forms of date and time ex...,Opensource,+,-
8,SentenceDetector*,Finds sentence bounds in raw text. Applies rul...,Opensource,+,-
9,DeepSentenceDetector*,Finds sentence bounds in raw text. Applies a N...,Opensource,+,-


In [14]:
text = 'Peter Parker is a nice guy and lives in New York.'

# spark is our session varaible we have created above as 'spark' = sparknlp.start() 
spark_df = spark.createDataFrame(([[text]])).toDF('text')
spark_df.show(truncate=False)

+-------------------------------------------------+
|text                                             |
+-------------------------------------------------+
|Peter Parker is a nice guy and lives in New York.|
+-------------------------------------------------+



In [17]:
spark_df = spark.createDataFrame(df)
spark_df.show(truncate=False)

+-----------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+------------------+---------------+
|Annotator              |Description                                                                                                                                                                                                |Version   |Annotator Approach|Annotator Model|
+-----------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+------------------+---------------+
|Tokenizer*             |Identifies tokens with tokenization open standards                                                                                                 

In [18]:
!wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jupyter/annotation/english/spark-nlp-basics/sample-sentences-en.txt

--2020-04-20 13:21:01--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jupyter/annotation/english/spark-nlp-basics/sample-sentences-en.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 284 [text/plain]
Saving to: ‘sample-sentences-en.txt’

sample-sentences-en 100%[===================>]     284  --.-KB/s    in 0s      

2020-04-20 13:21:01 (17.7 MB/s) - ‘sample-sentences-en.txt’ saved [284/284]



In [27]:
# When we use read.text --> it reads line by line
spark_df1 = spark.read.text('./sample-sentences-en.txt').toDF('text')
spark_df1.show(truncate=False)

+-----------------------------------------------------------------------------+
|text                                                                         |
+-----------------------------------------------------------------------------+
|Peter is a very good person.                                                 |
|My life in Russia is very interesting.                                       |
|John and Peter are brothers. However they don't support each other that much.|
|Lucas Nogal Dunbercker is no longer happy. He has a good car though.         |
|Europe is very culture rich. There are huge churches! and big houses!        |
+-----------------------------------------------------------------------------+



In [28]:
# Eger sadece 1 column secmek istersek
spark_df1.select('text').show(truncate=False)

+-----------------------------------------------------------------------------+
|text                                                                         |
+-----------------------------------------------------------------------------+
|Peter is a very good person.                                                 |
|My life in Russia is very interesting.                                       |
|John and Peter are brothers. However they don't support each other that much.|
|Lucas Nogal Dunbercker is no longer happy. He has a good car though.         |
|Europe is very culture rich. There are huge churches! and big houses!        |
+-----------------------------------------------------------------------------+



In [22]:
# The wholeTextFiles reads file as whole single document not line by line
textFiles = spark.sparkContext.wholeTextFiles("./*.txt", 4)
spark_df = textFiles.toDF(schema=['path', 'file_content'])
spark_df.show(truncate=30)

+------------------------------+------------------------------+
|                          path|                  file_content|
+------------------------------+------------------------------+
|file:/content/sample-senten...|Peter is a very good person...|
+------------------------------+------------------------------+



In [23]:
spark_df.select('file_content').take(1)

[Row(file_content="Peter is a very good person.\nMy life in Russia is very interesting.\nJohn and Peter are brothers. However they don't support each other that much.\nLucas Nogal Dunbercker is no longer happy. He has a good car though.\nEurope is very culture rich. There are huge churches! and big houses!")]

## Document Assambler

In [0]:
from sparknlp.base import *

In [29]:
documentAssembler = DocumentAssembler()\
.setInputCol('text')\
.setOutputCol('document')\
.setCleanupMode('shrink') # bosluklari, tab'leri vs temizler

doc_df = documentAssembler.transform(spark_df1)
doc_df.show(truncate=False)

+-----------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------+
|text                                                                         |document                                                                                                               |
+-----------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------+
|Peter is a very good person.                                                 |[[document, 0, 27, Peter is a very good person., [sentence -> 0], []]]                                                 |
|My life in Russia is very interesting.                                       |[[document, 0, 37, My life in Russia is very interesting., [sentence -> 0], []]]                                       |


In [30]:
doc_df.printSchema()
# as we can see we have the metadata here

root
 |-- text: string (nullable = true)
 |-- document: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)



In [31]:
doc_df.select('document.result', 'document.begin', 'document.end').show(truncate=False)
# each one returns a list type

+-------------------------------------------------------------------------------+-----+----+
|result                                                                         |begin|end |
+-------------------------------------------------------------------------------+-----+----+
|[Peter is a very good person.]                                                 |[0]  |[27]|
|[My life in Russia is very interesting.]                                       |[0]  |[37]|
|[John and Peter are brothers. However they don't support each other that much.]|[0]  |[76]|
|[Lucas Nogal Dunbercker is no longer happy. He has a good car though.]         |[0]  |[67]|
|[Europe is very culture rich. There are huge churches! and big houses!]        |[0]  |[68]|
+-------------------------------------------------------------------------------+-----+----+



In [34]:
# esas df'te document column uzerinde islem yapiyoruz o yuzden document.result olarak tanimlamaliyiz
doc_df.select("document.result").take(1)

[Row(result=['Peter is a very good person.'])]

In [36]:
### Expoloding teh dataframe (to view in much better nice view)

import pyspark.sql.functions as F

doc_df.withColumn(
    "tmp",
    F.explode('document'))\
    .select('tmp.*')\
    .show(truncate=False)

+-------------+-----+---+-----------------------------------------------------------------------------+---------------+----------+
|annotatorType|begin|end|result                                                                       |metadata       |embeddings|
+-------------+-----+---+-----------------------------------------------------------------------------+---------------+----------+
|document     |0    |27 |Peter is a very good person.                                                 |[sentence -> 0]|[]        |
|document     |0    |37 |My life in Russia is very interesting.                                       |[sentence -> 0]|[]        |
|document     |0    |76 |John and Peter are brothers. However they don't support each other that much.|[sentence -> 0]|[]        |
|document     |0    |67 |Lucas Nogal Dunbercker is no longer happy. He has a good car though.         |[sentence -> 0]|[]        |
|document     |0    |68 |Europe is very culture rich. There are huge churches! and 

## Sentence Detector

In [0]:
from sparknlp.annotator import *

# we feed the document column coming from Document Assembler

sentenceDetector = SentenceDetector().\
setInputCols(['document']).\
setOutputCol('sentences')

In [42]:
sent_df = sentenceDetector.transform(doc_df)
sent_df.show(truncate=False)

+-----------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|text                                                                         |document                                                                                                               |sentences                                                                                                                                                                                          |
+-----------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------+---------

In [44]:
# take works as head in pandas
sent_df.select('sentences').take(3)

[Row(sentences=[Row(annotatorType='document', begin=0, end=27, result='Peter is a very good person.', metadata={'sentence': '0'}, embeddings=[])]),
 Row(sentences=[Row(annotatorType='document', begin=0, end=37, result='My life in Russia is very interesting.', metadata={'sentence': '0'}, embeddings=[])]),
 Row(sentences=[Row(annotatorType='document', begin=0, end=27, result='John and Peter are brothers.', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='document', begin=29, end=76, result="However they don't support each other that much.", metadata={'sentence': '1'}, embeddings=[])])]

In [47]:
sample_text ="""The patient was prescribed 1 capsule of Advil for 5 days . He was seen by the endocrinology 
service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro 
with meals , and metformin 1000 mg two times a day . It was determined that all SGLT2 inhibitors should 
be discontinued indefinitely fro 3 months ."""
sample_text

'The patient was prescribed 1 capsule of Advil for 5 days . He was seen by the endocrinology \nservice and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro \nwith meals , and metformin 1000 mg two times a day . It was determined that all SGLT2 inhibitors should \nbe discontinued indefinitely fro 3 months .'

In [49]:
spark_df2 = spark.createDataFrame([[sample_text]]).toDF('text')
spark_df2.show(truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|text                                                                                                                                                                                                                                                                                                                                                   |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [54]:
doc_df1 = documentAssembler.transform(spark_df2)
sent_df = sentenceDetector.transform(doc_df1)
sent_df.show(truncate=40)

+----------------------------------------+----------------------------------------+----------------------------------------+
|                                    text|                                document|                               sentences|
+----------------------------------------+----------------------------------------+----------------------------------------+
|The patient was prescribed 1 capsule ...|[[document, 0, 339, The patient was p...|[[document, 0, 57, The patient was pr...|
+----------------------------------------+----------------------------------------+----------------------------------------+



In [56]:
sent_df.select('sentences.result').take(1)

[Row(result=['The patient was prescribed 1 capsule of Advil for 5 days .', 'He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , and metformin 1000 mg two times a day .', 'It was determined that all SGLT2 inhibitors should be discontinued indefinitely fro 3 months .'])]

In [57]:
sentenceDetector.setExplodeSentences(True)

SentenceDetector_18a10300d4df

In [64]:
sent_df = sentenceDetector.transform(doc_df1)
sent_df.select('sentences.result').show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|result                                                                                                                                                                                      |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[The patient was prescribed 1 capsule of Advil for 5 days .]                                                                                                                                |
|[He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , and metformin 1000 mg two times a day .]|
|[It was determined that all SGLT2 inhibitors

## Tokenizer

In [0]:
# #we can also use sentence in InputCols
tokenizer = Tokenizer()\
            .setInputCols(['document'])\
            .setOutputCol('token')

In [81]:
text2 = "Peter Parker (Spiderman) is a nice guy and live ins New York but has no e-mail! and he's gone so far"
text2.split()
# There are unwanted chars below which we do not want a part fo token such as () and !

['Peter',
 'Parker',
 '(Spiderman)',
 'is',
 'a',
 'nice',
 'guy',
 'and',
 'live',
 'ins',
 'New',
 'York',
 'but',
 'has',
 'no',
 'e-mail!',
 'and',
 "he's",
 'gone',
 'so',
 'far']

In [0]:
spark_df = spark.createDataFrame([[text2]]).toDF('text')

In [83]:
doc_df = documentAssembler.transform(spark_df)
token_df = tokenizer.fit(doc_df).transform(doc_df)
token_df.show(truncate=100)

+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+
|                                                                                                text|                                                                                            document|                                                                                               token|
+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+
|Peter Parker (Spiderman) is a nice guy and live ins New York but has no e-mail! and 

In [84]:
token_df.select('token.result').take(1)

[Row(result=['Peter', 'Parker', '(', 'Spiderman', ')', 'is', 'a', 'nice', 'guy', 'and', 'live', 'ins', 'New', 'York', 'but', 'has', 'no', 'e-mail', '!', 'and', "he's", 'gone', 'so', 'far'])]

In [0]:
# Eger tokenizer i custome etmek istersek

tokenizer = Tokenizer()\
            .setInputCols(['document'])\
            .setOutputCol('token')\
            .setSplitChars(['-'])\
            .setContextChars(['(', ')', '?', '!'])\
            .addException('New York')

In [87]:
token_df = tokenizer.fit(doc_df).transform(token_df)
token_df.select('token.result').take(1)

[Row(result=['Peter', 'Parker', '(', 'Spiderman', ')', 'is', 'a', 'nice', 'guy', 'and', 'live', 'ins', 'New York', 'but', 'has', 'no', 'e', 'mail', '!', 'and', "he's", 'gone', 'so', 'far'])]

In [0]:
# # Pay Attention to () for Spiderman
tokenizer33 = Tokenizer()\
            .setInputCols(['document'])\
            .setOutputCol('token')\
            .setSplitChars(['-'])\
            .setContextChars(['?', '!'])\
            .addException('New York')

In [104]:
token_df = tokenizer33.fit(doc_df).transform(token_df)
token_df.select('token.result').take(1)

[Row(result=['Peter', 'Parker', '(Spiderman)', 'is', 'a', 'nice', 'guy', 'and', 'live', 'ins', 'New York', 'but', 'has', 'no', 'e', 'mail', '!', 'and', "he's", 'gone', 'so', 'far'])]

In [0]:
### Let's Combine Everything in a Pipeline

In [0]:
from pyspark.ml import Pipeline

documentAssembler = DocumentAssembler()\
                    .setInputCol('text')\
                    .setOutputCol('document')

sentenceDetector = SentenceDetector()\
                    .setInputCols(['document'])\
                    .setOutputCol('sentences')

tokenizer = Tokenizer()\
            .setInputCols(['sentences'])\
            .setOutputCol('token')
        
nlPipeline = Pipeline(stages=[
                              documentAssembler,
                              sentenceDetector,
                              tokenizer
])

empty_df = spark.createDataFrame([['']]).toDF('text')

pipelineModel = nlPipeline.fit(empty_df)

In [108]:
spark_df = spark.read.text('./sample-sentences-en.txt').toDF('text')
spark_df.show(truncate=False)

+-----------------------------------------------------------------------------+
|text                                                                         |
+-----------------------------------------------------------------------------+
|Peter is a very good person.                                                 |
|My life in Russia is very interesting.                                       |
|John and Peter are brothers. However they don't support each other that much.|
|Lucas Nogal Dunbercker is no longer happy. He has a good car though.         |
|Europe is very culture rich. There are huge churches! and big houses!        |
+-----------------------------------------------------------------------------+



In [111]:
result = pipelineModel.transform(spark_df)
result.show(truncate=20)

+--------------------+--------------------+--------------------+--------------------+
|                text|            document|           sentences|               token|
+--------------------+--------------------+--------------------+--------------------+
|Peter is a very g...|[[document, 0, 27...|[[document, 0, 27...|[[token, 0, 4, Pe...|
|My life in Russia...|[[document, 0, 37...|[[document, 0, 37...|[[token, 0, 1, My...|
|John and Peter ar...|[[document, 0, 76...|[[document, 0, 27...|[[token, 0, 3, Jo...|
|Lucas Nogal Dunbe...|[[document, 0, 67...|[[document, 0, 41...|[[token, 0, 4, Lu...|
|Europe is very cu...|[[document, 0, 68...|[[document, 0, 27...|[[token, 0, 5, Eu...|
+--------------------+--------------------+--------------------+--------------------+



In [112]:
result.printSchema()

root
 |-- text: string (nullable = true)
 |-- document: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- sentences: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = tru

In [113]:
result.select('sentences.result').take(3)

[Row(result=['Peter is a very good person.']),
 Row(result=['My life in Russia is very interesting.']),
 Row(result=['John and Peter are brothers.', "However they don't support each other that much."])]

In [115]:
result.select('token').take(3)[2]

Row(token=[Row(annotatorType='token', begin=0, end=3, result='John', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=5, end=7, result='and', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=9, end=13, result='Peter', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=15, end=17, result='are', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=19, end=26, result='brothers', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=27, end=27, result='.', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=29, end=35, result='However', metadata={'sentence': '1'}, embeddings=[]), Row(annotatorType='token', begin=37, end=40, result='they', metadata={'sentence': '1'}, embeddings=[]), Row(annotatorType='token', begin=42, end=46, result="don't", metadata={'sentence': '1'}, embeddings=[]), Row(annotatorType='token', begin=48, end=54, result='s

In [116]:
result.select('token.result').take(3)

[Row(result=['Peter', 'is', 'a', 'very', 'good', 'person', '.']),
 Row(result=['My', 'life', 'in', 'Russia', 'is', 'very', 'interesting', '.']),
 Row(result=['John', 'and', 'Peter', 'are', 'brothers', '.', 'However', 'they', "don't", 'support', 'each', 'other', 'that', 'much', '.'])]

In [123]:
''.join([i for i in "My name is @# Serdar." if i.isalnum() or i==' '])

'My name is  Serdar'

## Normalizer

In [118]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [0]:
from sparknlp.annotator import *

normalizer = Normalizer()\
            .setInputCols(['token'])\
            .setOutputCol('normalized')\
            .setLowercase(True)\
            .setCleanupPatterns(["[^\w\d\s]"]) #remove punctuations and keep alphanumerics only
            # if we do not set ClenaupPatterns it till only keep alphabet letters [^A-Za-z]

In [0]:
# We're removing sentence annotator


documentAssembler = DocumentAssembler()\
.setInputCol("text")\
.setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

nlPipeline = Pipeline(stages=[
                              documentAssembler,
                              tokenizer,
                              normalizer
])
empty_df = spark.createDataFrame([['']]).toDF('text')

pipelineModel = nlPipeline.fit(empty_df)

In [132]:
result = pipelineModel.transform(spark_df)
result.show(truncate=True)

+--------------------+--------------------+--------------------+--------------------+
|                text|            document|               token|          normalized|
+--------------------+--------------------+--------------------+--------------------+
|Peter is a very g...|[[document, 0, 27...|[[token, 0, 4, Pe...|[[token, 0, 4, pe...|
|My life in Russia...|[[document, 0, 37...|[[token, 0, 1, My...|[[token, 0, 1, my...|
|John and Peter ar...|[[document, 0, 76...|[[token, 0, 3, Jo...|[[token, 0, 3, jo...|
|Lucas Nogal Dunbe...|[[document, 0, 67...|[[token, 0, 4, Lu...|[[token, 0, 4, lu...|
|Europe is very cu...|[[document, 0, 68...|[[token, 0, 5, Eu...|[[token, 0, 5, eu...|
+--------------------+--------------------+--------------------+--------------------+



In [133]:
result.select('normalized.result').take(3)

[Row(result=['peter', 'is', 'a', 'very', 'good', 'person']),
 Row(result=['my', 'life', 'in', 'russia', 'is', 'very', 'interesting']),
 Row(result=['john', 'and', 'peter', 'are', 'brothers', 'however', 'they', 'dont', 'support', 'each', 'other', 'that', 'much'])]

## Stopwrods

In [0]:
stopwords_cleaner = StopWordsCleaner()\
                    .setInputCols('token')\
                    .setOutputCol('cleanTokens')\
                    .setCaseSensitive(False)
                    #.setStopWords(['no', 'without']) custom stopwords we can use but it will overwrite existing stopwords and will ONLY use this one
                    # if we want to  use normalized ones and just want to use from normalised use "normalized" in InputCols

documentAssembler = DocumentAssembler()\
.setInputCol("text")\
.setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

nlPipeline = Pipeline(stages=[
                              documentAssembler,
                              tokenizer,
                              stopwords_cleaner
])

empty_df = spark.createDataFrame([['']]).toDF('text')

pipelineModel = nlPipeline.fit(empty_df)

In [147]:
result = pipelineModel.transform(spark_df)
result.show(truncate=True)

+--------------------+--------------------+--------------------+--------------------+
|                text|            document|               token|         cleanTokens|
+--------------------+--------------------+--------------------+--------------------+
|Peter is a very g...|[[document, 0, 27...|[[token, 0, 4, Pe...|[[token, 0, 4, Pe...|
|My life in Russia...|[[document, 0, 37...|[[token, 0, 1, My...|[[token, 3, 6, li...|
|John and Peter ar...|[[document, 0, 76...|[[token, 0, 3, Jo...|[[token, 0, 3, Jo...|
|Lucas Nogal Dunbe...|[[document, 0, 67...|[[token, 0, 4, Lu...|[[token, 0, 4, Lu...|
|Europe is very cu...|[[document, 0, 68...|[[token, 0, 5, Eu...|[[token, 0, 5, Eu...|
+--------------------+--------------------+--------------------+--------------------+



In [149]:
result.select('cleanTokens.result').take(3)
# We did not include Normalizer so we might see punctuations and uppercase. If we dont want, se should incluce in pipeline

[Row(result=['Peter', 'good', 'person', '.']),
 Row(result=['life', 'Russia', 'interesting', '.']),
 Row(result=['John', 'Peter', 'brothers', '.', 'However', 'support', 'much', '.'])]

## Token Assembler

In [150]:
documentAssembler = DocumentAssembler()\
.setInputCol("text")\
.setOutputCol("document")

sentenceDetector = SentenceDetector().\
    setInputCols(['document']).\
    setOutputCol('sentences')

tokenizer = Tokenizer() \
    .setInputCols(["sentences"]) \
    .setOutputCol("token")

normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normalized")\
    .setLowercase(False)\

stopwords_cleaner = StopWordsCleaner()\
      .setInputCols("normalized")\
      .setOutputCol("cleanTokens")\
      .setCaseSensitive(False)\

tokenassembler = TokenAssembler()\
    .setInputCols(["cleanTokens"]) \
    .setOutputCol("clean_text")


nlpPipeline = Pipeline(stages=[
     documentAssembler,
    sentenceDetector,
     tokenizer,
     normalizer,
     stopwords_cleaner,
     tokenassembler
 ])


empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

result = pipelineModel.transform(spark_df)

result.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|           sentences|               token|          normalized|         cleanTokens|          clean_text|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Peter is a very g...|[[document, 0, 27...|[[document, 0, 27...|[[token, 0, 4, Pe...|[[token, 0, 4, Pe...|[[token, 0, 4, Pe...|[[document, 0, 16...|
|My life in Russia...|[[document, 0, 37...|[[document, 0, 37...|[[token, 0, 1, My...|[[token, 0, 1, My...|[[token, 3, 6, li...|[[document, 3, 25...|
|John and Peter ar...|[[document, 0, 76...|[[document, 0, 27...|[[token, 0, 3, Jo...|[[token, 0, 3, Jo...|[[token, 0, 3, Jo...|[[document, 0, 18...|
|Lucas Nogal Dunbe...|[[document, 0, 67...|[[document, 0, 41...|[[token, 0, 4, Lu...|[[token, 0, 4, Lu...|

In [151]:
result.select(['text', 'clean_text.result']).show(truncate=False)

+-----------------------------------------------------------------------------+------------------------------------------------------+
|text                                                                         |result                                                |
+-----------------------------------------------------------------------------+------------------------------------------------------+
|Peter is a very good person.                                                 |[Peter good person]                                   |
|My life in Russia is very interesting.                                       |[life Russia interesting]                             |
|John and Peter are brothers. However they don't support each other that much.|[John Peter brothers, However dont support much]      |
|Lucas Nogal Dunbercker is no longer happy. He has a good car though.         |[Lucas Nogal Dunbercker longer happy, good car though]|
|Europe is very culture rich. There are huge churches! 

In [153]:
import pyspark.sql.functions as F

result.withColumn(
    "tmp",
    F.explode('clean_text'))\
    .select("tmp.*").select('begin', 'end', 'result', 'metadata.sentence').show(truncate=False)

+-----+---+-----------------------------------+--------+
|begin|end|result                             |sentence|
+-----+---+-----------------------------------+--------+
|0    |16 |Peter good person                  |0       |
|3    |25 |life Russia interesting            |0       |
|0    |18 |John Peter brothers                |0       |
|29   |53 |However dont support much          |1       |
|0    |34 |Lucas Nogal Dunbercker longer happy|0       |
|52   |66 |good car though                    |1       |
|0    |18 |Europe culture rich                |0       |
|39   |51 |huge churches                      |1       |
|58   |67 |big houses                         |2       |
+-----+---+-----------------------------------+--------+



In [154]:
# if we hadn't used Sentence Detector, this would be what we got. (tokenizer gets document instead of sentences column)

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")


nlpPipeline = Pipeline(stages=[
     documentAssembler,
     tokenizer,
     normalizer,
     stopwords_cleaner,
     tokenassembler
 ])


empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

result = pipelineModel.transform(spark_df)

result.select('text', 'clean_text.result').show(truncate=False)

+-----------------------------------------------------------------------------+-----------------------------------------------------+
|text                                                                         |result                                               |
+-----------------------------------------------------------------------------+-----------------------------------------------------+
|Peter is a very good person.                                                 |[Peter good person]                                  |
|My life in Russia is very interesting.                                       |[life Russia interesting]                            |
|John and Peter are brothers. However they don't support each other that much.|[John Peter brothers However dont support much]      |
|Lucas Nogal Dunbercker is no longer happy. He has a good car though.         |[Lucas Nogal Dunbercker longer happy good car though]|
|Europe is very culture rich. There are huge churches! and big

In [155]:
result.withColumn(
    "tmp",
    F.explode('clean_text'))\
    .select("tmp.*").select('begin', 'end', 'result', 'metadata.sentence').show(truncate=False)

# We get all sentences in a single lien and all sentences assigend as 0, there si no other sentence

+-----+---+---------------------------------------------------+--------+
|begin|end|result                                             |sentence|
+-----+---+---------------------------------------------------+--------+
|0    |16 |Peter good person                                  |0       |
|3    |25 |life Russia interesting                            |0       |
|0    |44 |John Peter brothers However dont support much      |0       |
|0    |50 |Lucas Nogal Dunbercker longer happy good car though|0       |
|0    |43 |Europe culture rich huge churches big houses       |0       |
+-----+---+---------------------------------------------------+--------+



## Stemmer

In [0]:
stemmer = Stemmer() \
    .setInputCols(["token"]) \
    .setOutputCol("stem")

In [0]:

documentAssembler = DocumentAssembler()\
.setInputCol("text")\
.setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

nlpPipeline = Pipeline(stages=[
 documentAssembler, 
 tokenizer,
 stemmer
 ])

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

In [158]:

result = pipelineModel.transform(spark_df)

result.show()

+--------------------+--------------------+--------------------+--------------------+
|                text|            document|               token|                stem|
+--------------------+--------------------+--------------------+--------------------+
|Peter is a very g...|[[document, 0, 27...|[[token, 0, 4, Pe...|[[token, 0, 4, pe...|
|My life in Russia...|[[document, 0, 37...|[[token, 0, 1, My...|[[token, 0, 1, my...|
|John and Peter ar...|[[document, 0, 76...|[[token, 0, 3, Jo...|[[token, 0, 3, jo...|
|Lucas Nogal Dunbe...|[[document, 0, 67...|[[token, 0, 4, Lu...|[[token, 0, 4, lu...|
|Europe is very cu...|[[document, 0, 68...|[[token, 0, 5, Eu...|[[token, 0, 5, eu...|
+--------------------+--------------------+--------------------+--------------------+



In [159]:
result.select('stem.result').show(truncate=False)

+-------------------------------------------------------------------------------------------+
|result                                                                                     |
+-------------------------------------------------------------------------------------------+
|[peter, i, a, veri, good, person, .]                                                       |
|[my, life, in, russia, i, veri, interest, .]                                               |
|[john, and, peter, ar, brother, ., howev, thei, don't, support, each, other, that, much, .]|
|[luca, nogal, dunberck, i, no, longer, happi, ., he, ha, a, good, car, though, .]          |
|[europ, i, veri, cultur, rich, ., there, ar, huge, church, !, and, big, hous, !]           |
+-------------------------------------------------------------------------------------------+



In [160]:
import pyspark.sql.functions as F

result_df = result.select(F.explode(F.arrays_zip('token.result', 'stem.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("token"),
        F.expr("cols['1']").alias("stem")).toPandas()

result_df.head(10)

,token,stem
0,Peter,peter
1,is,i
2,a,a
3,very,veri
4,good,good
5,person,person
6,.,.
7,My,my
8,life,life
9,in,in


## Lemmatizer

In [161]:
!wget https://raw.githubusercontent.com/mahavivo/vocabulary/master/lemmas/AntBNC_lemmas_ver_001.txt

--2020-04-20 16:01:21--  https://raw.githubusercontent.com/mahavivo/vocabulary/master/lemmas/AntBNC_lemmas_ver_001.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1348552 (1.3M) [text/plain]
Saving to: ‘AntBNC_lemmas_ver_001.txt’

AntBNC_lemmas_ver_0 100%[===================>]   1.29M  --.-KB/s    in 0.01s   

2020-04-20 16:01:21 (128 MB/s) - ‘AntBNC_lemmas_ver_001.txt’ saved [1348552/1348552]



In [0]:
lemmatizer = Lemmatizer() \
    .setInputCols(["token"]) \
    .setOutputCol("lemma") \
    .setDictionary("./AntBNC_lemmas_ver_001.txt", value_delimiter ="\t", key_delimiter = "->")

In [0]:
documentAssembler = DocumentAssembler()\
.setInputCol("text")\
.setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

stemmer = Stemmer() \
    .setInputCols(["token"]) \
    .setOutputCol("stem")

nlpPipeline = Pipeline(stages=[
 documentAssembler, 
 tokenizer,
 stemmer,
 lemmatizer
 ])

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

In [164]:
result = pipelineModel.transform(spark_df)

result.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|               token|                stem|               lemma|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|Peter is a very g...|[[document, 0, 27...|[[token, 0, 4, Pe...|[[token, 0, 4, pe...|[[token, 0, 4, Pe...|
|My life in Russia...|[[document, 0, 37...|[[token, 0, 1, My...|[[token, 0, 1, my...|[[token, 0, 1, My...|
|John and Peter ar...|[[document, 0, 76...|[[token, 0, 3, Jo...|[[token, 0, 3, jo...|[[token, 0, 3, Jo...|
|Lucas Nogal Dunbe...|[[document, 0, 67...|[[token, 0, 4, Lu...|[[token, 0, 4, lu...|[[token, 0, 4, Lu...|
|Europe is very cu...|[[document, 0, 68...|[[token, 0, 5, Eu...|[[token, 0, 5, eu...|[[token, 0, 5, Eu...|
+--------------------+--------------------+--------------------+--------------------+--------------------+



In [165]:
result.select('lemma.result').show(truncate=False)

+---------------------------------------------------------------------------------------------+
|result                                                                                       |
+---------------------------------------------------------------------------------------------+
|[Peter, be, a, very, good, person, .]                                                        |
|[My, life, in, Russia, be, very, interest, .]                                                |
|[John, and, Peter, be, brother, ., However, they, don't, support, each, other, that, much, .]|
|[Lucas, Nogal, Dunbercker, be, no, long, happy, ., He, have, a, good, car, though, .]        |
|[Europe, be, very, culture, rich, ., There, be, huge, church, !, and, big, house, !]         |
+---------------------------------------------------------------------------------------------+



In [166]:

result_df = result.select(F.explode(F.arrays_zip('token.result', 'stem.result',  'lemma.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("token"),
        F.expr("cols['1']").alias("stem"),
        F.expr("cols['2']").alias("lemma")).toPandas()

result_df.head(10)

,token,stem,lemma
0,Peter,peter,Peter
1,is,i,be
2,a,a,a
3,very,veri,very
4,good,good,good
5,person,person,person
6,.,.,.
7,My,my,My
8,life,life,life
9,in,in,in


## N-Gram

In [168]:
ngrams_cum = NGramGenerator() \
            .setInputCols(["token"]) \
            .setOutputCol("ngrams") \
            .setN(3) \
            .setEnableCumulative(True)\
            .setDelimiter("_") # Default is space
    
# .setN(3) means, take bigrams and trigrams.

nlpPipeline = Pipeline(stages=[
 documentAssembler, 
 tokenizer,
 ngrams_cum
 ])

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

result = pipelineModel.transform(spark_df)

result.select('ngrams.result').show(truncate=200)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                                                                                                                                                                                  result|
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                    [Peter, is, a, very, good, person, ., Peter_is, is_a, a_very, very_good, good_person, person_., Peter_is_a, is_a_very, a_very_good, very_good_person, good_person_.]|
|[My, life, in, Russia, is, very, interesting, ., My_life, life_in, in_Russia, Russia_is, is_very, very_interesting, interesting_., My_life_in, life_in_Russia, in_Russia_is, Russia_is_very

## Text Matcher

In [169]:
! wget 	https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/pubmed/pubmed-sample.csv

--2020-04-20 16:11:48--  https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/pubmed/pubmed-sample.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.110.13
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.110.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10484510 (10.0M) [text/csv]
Saving to: ‘pubmed-sample.csv’

pubmed-sample.csv   100%[===================>]  10.00M  16.8MB/s    in 0.6s    

2020-04-20 16:11:49 (16.8 MB/s) - ‘pubmed-sample.csv’ saved [10484510/10484510]



In [174]:
!head pubmed-sample.csv

AB,TI
"The human KCNJ9 (Kir 3.3, GIRK3) is a member of the G-protein-activated inwardly rectifying potassium (GIRK) channel family. Here we describe the genomicorganization of the KCNJ9 locus on chromosome 1q21-23 as a candidate gene forType II diabetes mellitus in the Pima Indian population. The gene spansapproximately 7.6 kb and contains one noncoding and two coding exons separated byapproximately 2.2 and approximately 2.6 kb introns, respectively. We identified14 single nucleotide polymorphisms (SNPs), including one that predicts aVal366Ala substitution, and an 8 base-pair (bp) insertion/deletion. Ourexpression studies revealed the presence of the transcript in various humantissues including pancreas, and two major insulin-responsive tissues: fat andskeletal muscle. The characterization of the KCNJ9 gene should facilitate furtherstudies on the function of the KCNJ9 protein and allow evaluation of thepotential role of the locus in Type II diabetes.","Genomic structure and expression 

In [175]:
import pyspark.sql.functions as F

pubMedDF = spark.read\
                .option("header", "true")\
                .csv("./pubmed-sample.csv")\
                .filter("AB IS NOT null")\
                .withColumnRenamed("AB", "text")\
                .drop("TI")

pubMedDF.show(truncate=100)

+----------------------------------------------------------------------------------------------------+
|                                                                                                text|
+----------------------------------------------------------------------------------------------------+
|The human KCNJ9 (Kir 3.3, GIRK3) is a member of the G-protein-activated inwardly rectifying potas...|
|BACKGROUND: At present, it is one of the most important issues for the treatment of breast cancer...|
|OBJECTIVE: To investigate the relationship between preoperative atrialfibrillation and early and ...|
|Combined EEG/fMRI recording has been used to localize the generators of EEGevents and to identify...|
|Kohlschutter syndrome is a rare neurodegenerative disorder presenting withintractable seizures, d...|
|Statistical analysis of neuroimages is commonly approached with intergroupcomparisons made by rep...|
|The synthetic DOX-LNA conjugate was characterized by proton nuclear magn

In [0]:
# let's create a file which will includes the entities we want to find
# it will be stored in content directory
# It is used for the EXACT WORD/WORDS we're looking for

entities = ['KCNJ9', 'GIRK', 'diabates mellitus', 'nucleotide polymorphisms']

with open('clinical_entities.txt', 'w') as f:
    for i in entities:
        f.write(i+'\n')

In [0]:
entity_extractor = TextMatcher()\
                    .setInputCols(['document', 'token'])\
                    .setOutputCol('matched_entities')\
                    .setEntities('clinical_entities.txt')

In [0]:
nlpPipeline = Pipeline(stages=[
 documentAssembler, 
 tokenizer,
 entity_extractor
 ])

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

In [182]:
pubMedDF.count()
# it has 7.5K rows

7537

In [0]:
result = pipelineModel.transform(pubMedDF.limit(10))

In [183]:
pubMedDF.select('text').take(1)

[Row(text='The human KCNJ9 (Kir 3.3, GIRK3) is a member of the G-protein-activated inwardly rectifying potassium (GIRK) channel family. Here we describe the genomicorganization of the KCNJ9 locus on chromosome 1q21-23 as a candidate gene forType II diabetes mellitus in the Pima Indian population. The gene spansapproximately 7.6 kb and contains one noncoding and two coding exons separated byapproximately 2.2 and approximately 2.6 kb introns, respectively. We identified14 single nucleotide polymorphisms (SNPs), including one that predicts aVal366Ala substitution, and an 8 base-pair (bp) insertion/deletion. Ourexpression studies revealed the presence of the transcript in various humantissues including pancreas, and two major insulin-responsive tissues: fat andskeletal muscle. The characterization of the KCNJ9 gene should facilitate furtherstudies on the function of the KCNJ9 protein and allow evaluation of thepotential role of the locus in Type II diabetes.')]

In [181]:
result.select('matched_entities.result').take(1)

[Row(result=['KCNJ9', 'GIRK', 'KCNJ9', 'nucleotide polymorphisms', 'KCNJ9', 'KCNJ9'])]

In [184]:
result_df = result.select(F.explode(F.arrays_zip('matched_entities.result', 'matched_entities.begin',  'matched_entities.end')).alias("cols")) \
.select(F.expr("cols['0']").alias("matched_entities"),
        F.expr("cols['1']").alias("begin"),
        F.expr("cols['2']").alias("end")).toPandas()

result_df.head(10)

,matched_entities,begin,end
0,KCNJ9,10,14
1,GIRK,103,106
2,KCNJ9,173,177
3,nucleotide polymorphisms,471,494
4,KCNJ9,801,805
5,KCNJ9,868,872


## RegexMatcher

In [0]:
rules = '''
renal\s\w+, followed by 'renal'
cardiac\s\w+, followed by 'cardiac'
'''

with open('regex_rules.txt', 'w') as f:
    
    f.write(rules)

In [186]:
documentAssembler = DocumentAssembler()\
.setInputCol("text")\
.setOutputCol("document")

regex_matcher = RegexMatcher()\
    .setInputCols('document')\
    .setStrategy("MATCH_ALL")\
    .setOutputCol("regex_matches")\
    .setExternalRules(path='./regex_rules.txt', delimiter=',')
    

nlpPipeline = Pipeline(stages=[
 documentAssembler, 
 regex_matcher
 ])

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

match_df = pipelineModel.transform(pubMedDF)

match_df.select('regex_matches.result').take(3)

[Row(result=[]),
 Row(result=[]),
 Row(result=['renal failure', 'cardiac surgery', 'cardiac surgery', 'cardiac surgical'])]

In [187]:
# Sadece regex iceren satirlari gormek istersek

match_df.select('text','regex_matches.result')\
.toDF('text','matches').filter(F.size('matches')>1)\
.show(truncate=50)

+--------------------------------------------------+--------------------------------------------------+
|                                              text|                                           matches|
+--------------------------------------------------+--------------------------------------------------+
|OBJECTIVE: To investigate the relationship betw...|[renal failure, cardiac surgery, cardiac surger...|
|Aging is a physiological process that causes st...|                     [renal function, renal aging]|
|BACKGROUND: Left-sided inferior vena cava (IVC)...|          [renal surgery, renal mass, renal veins]|
|A successful kidney transplantation (Tx) offers...|                         [renal failure, renal Tx]|
|The transcription factor scleraxis has been imp...|[cardiac valves, cardiac fibroblasts, cardiac c...|
|Historically, the only surgical option for remo...|                        [renal mass, renal masses]|
|OBJECTIVES: To examine the clinical effectivene...|   [renal tr

In [188]:
match_df.printSchema()

root
 |-- text: string (nullable = true)
 |-- document: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- regex_matches: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull =

## Finisher

In [0]:
# It removes previous columns which is defined in pipeline and shows only the one which is defined in finisher
# Finisher: Once we have our NLP pipeline ready to go, we might want to use our annotation results somewhere else where it is easy to use. 
# The Finisher outputs annotation(s) values into a string.

In [0]:
finisher = Finisher()\
        .setInputCols(['regex_matches'])\
        .setIncludeMetadata(False) # set False to remove metadata

In [193]:
nlpPipeline = Pipeline(stages=[
 documentAssembler, 
 regex_matcher,
 finisher
 ])

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

match_df = pipelineModel.transform(pubMedDF)

match_df.show(truncate = 50)

+--------------------------------------------------+--------------------------------------------------+
|                                              text|                            finished_regex_matches|
+--------------------------------------------------+--------------------------------------------------+
|The human KCNJ9 (Kir 3.3, GIRK3) is a member of...|                                                []|
|BACKGROUND: At present, it is one of the most i...|                                                []|
|OBJECTIVE: To investigate the relationship betw...|[renal failure, cardiac surgery, cardiac surger...|
|Combined EEG/fMRI recording has been used to lo...|                                                []|
|Kohlschutter syndrome is a rare neurodegenerati...|                                                []|
|Statistical analysis of neuroimages is commonly...|                                                []|
|The synthetic DOX-LNA conjugate was characteriz...|            

In [194]:
match_df.printSchema()
# it is very simple now

root
 |-- text: string (nullable = true)
 |-- finished_regex_matches: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [196]:
# If we want to filter out the empty matches

match_df.filter(F.size('finished_regex_matches')>=1).show(truncate = 50)

+--------------------------------------------------+--------------------------------------------------+
|                                              text|                            finished_regex_matches|
+--------------------------------------------------+--------------------------------------------------+
|OBJECTIVE: To investigate the relationship betw...|[renal failure, cardiac surgery, cardiac surger...|
|Successful renal transplantation is associated ...|                           [renal transplantation]|
|Although inhaled and intranasal corticosteroids...|                               [renal suppression]|
|Aging is a physiological process that causes st...|                     [renal function, renal aging]|
|Tight junctions form the major paracellular bar...|                                     [renal tight]|
|BACKGROUND: Left-sided inferior vena cava (IVC)...|          [renal surgery, renal mass, renal veins]|
|A successful kidney transplantation (Tx) offers...|            

## LightPipeline

LightPipelines are Spark NLP specific Pipelines, equivalent to Spark ML Pipeline, but meant to **deal with smaller amounts of data**. They’re useful working with small datasets, debugging results, or when running either training or prediction from an API that serves one-off requests.

Spark NLP LightPipelines are Spark ML pipelines converted into a single machine but the multi-threaded task, becoming more than 10x times faster for smaller amounts of data (small is relative, but 50k sentences are roughly a good maximum). To use them, we simply plug in a trained (fitted) pipeline and then annotate a plain text. We don't even need to convert the input text to DataFrame in order to feed it into a pipeline that's accepting DataFrame as an input in the first place. This feature would be quite useful when it comes to getting a prediction for a few lines of text from a trained ML model.

It is nearly 20x faster than using Spark ML Pipeline

In [0]:
# Note we re-run the pipelineModel which inludes 'lemma', 'stem' etc above so the the very last one we're processing which is coming 
# from regex_matches. Please pya attention that

In [0]:
from sparknlp.base import LightPipeline

light_model = LightPipeline(pipelineModel)

light_result = light_model.annotate("John and Peter are brothers. However they don't support each other that much")

In [205]:
light_result.keys()

dict_keys(['document', 'token', 'stem', 'lemma'])

In [208]:
list(zip(light_result['token'], light_result['stem'], light_result['lemma']))

[('John', 'john', 'John'),
 ('and', 'and', 'and'),
 ('Peter', 'peter', 'Peter'),
 ('are', 'ar', 'be'),
 ('brothers', 'brother', 'brother'),
 ('.', '.', '.'),
 ('However', 'howev', 'However'),
 ('they', 'thei', 'they'),
 ("don't", "don't", "don't"),
 ('support', 'support', 'support'),
 ('each', 'each', 'each'),
 ('other', 'other', 'other'),
 ('that', 'that', 'that'),
 ('much', 'much', 'much')]

In [210]:
light_result = light_model.fullAnnotate("John and Peter are brothers. However they don't support each other that much")
light_result

[{'document': [Annotation(document, 0, 75, John and Peter are brothers. However they don't support each other that much, {})],
  'lemma': [Annotation(token, 0, 3, John, {'sentence': '0'}),
   Annotation(token, 5, 7, and, {'sentence': '0'}),
   Annotation(token, 9, 13, Peter, {'sentence': '0'}),
   Annotation(token, 15, 17, be, {'sentence': '0'}),
   Annotation(token, 19, 26, brother, {'sentence': '0'}),
   Annotation(token, 27, 27, ., {'sentence': '0'}),
   Annotation(token, 29, 35, However, {'sentence': '0'}),
   Annotation(token, 37, 40, they, {'sentence': '0'}),
   Annotation(token, 42, 46, don't, {'sentence': '0'}),
   Annotation(token, 48, 54, support, {'sentence': '0'}),
   Annotation(token, 56, 59, each, {'sentence': '0'}),
   Annotation(token, 61, 65, other, {'sentence': '0'}),
   Annotation(token, 67, 70, that, {'sentence': '0'}),
   Annotation(token, 72, 75, much, {'sentence': '0'})],
  'stem': [Annotation(token, 0, 3, john, {'sentence': '0'}),
   Annotation(token, 5, 7, and,

In [0]:
### First NoteBook Can Be Seen From Here to Below

## Using Pretrained Pipelines

https://github.com/JohnSnowLabs/spark-nlp-models

In [0]:
from sparknlp.pretrained import PretrainedPipeline

In [2]:
pipeline = PretrainedPipeline('explain_document_ml', lang='en')

NameError: ignored

In [0]:
testDoc = """French author who helped pioner the science-fiction genre. \
Verne wrate about space, air, and underwater travel before navigable aircrast and \
practical submarines were invented, and before any means of space travel had been devised."""

In [0]:
testDoc

'French author who helped pioner the science-fiction genre. Verne wrate about space, air, and underwater travel before navigable aircrast and practical submarines were invented, and before any means of space travel had been devised.'

In [0]:
%%time

result = pipeline.annotate(testDoc)

CPU times: user 34.9 ms, sys: 9.77 ms, total: 44.6 ms
Wall time: 2.26 s


In [0]:
result.keys()

dict_keys(['document', 'spell', 'pos', 'lemmas', 'token', 'stems', 'sentence'])

In [0]:
result['sentence']

['French author who helped pioner the science-fiction genre.',
 'Verne wrate about space, air, and underwater travel before navigable aircrast and practical submarines were invented, and before any means of space travel had been devised.']

In [0]:
result['token']

['French',
 'author',
 'who',
 'helped',
 'pioner',
 'the',
 'science-fiction',
 'genre',
 '.',
 'Verne',
 'wrate',
 'about',
 'space',
 ',',
 'air',
 ',',
 'and',
 'underwater',
 'travel',
 'before',
 'navigable',
 'aircrast',
 'and',
 'practical',
 'submarines',
 'were',
 'invented',
 ',',
 'and',
 'before',
 'any',
 'means',
 'of',
 'space',
 'travel',
 'had',
 'been',
 'devised',
 '.']

In [0]:
list(zip(result['token'], result['pos']))

[('French', 'JJ'),
 ('author', 'NN'),
 ('who', 'WP'),
 ('helped', 'VBD'),
 ('pioner', 'NN'),
 ('the', 'DT'),
 ('science-fiction', 'NN'),
 ('genre', 'NN'),
 ('.', '.'),
 ('Verne', 'NNP'),
 ('wrate', 'VBD'),
 ('about', 'IN'),
 ('space', 'NN'),
 (',', ','),
 ('air', 'NN'),
 (',', ','),
 ('and', 'CC'),
 ('underwater', 'JJ'),
 ('travel', 'NN'),
 ('before', 'IN'),
 ('navigable', 'JJ'),
 ('aircrast', 'NN'),
 ('and', 'CC'),
 ('practical', 'JJ'),
 ('submarines', 'NNS'),
 ('were', 'VBD'),
 ('invented', 'VBN'),
 (',', ','),
 ('and', 'CC'),
 ('before', 'IN'),
 ('any', 'DT'),
 ('means', 'NNS'),
 ('of', 'IN'),
 ('space', 'NN'),
 ('travel', 'NN'),
 ('had', 'VBD'),
 ('been', 'VBN'),
 ('devised', 'VBN'),
 ('.', '.')]

In [0]:
list(zip(result['token'], result['lemmas'], result['stems'], result['spell']))

[('French', 'French', 'french', 'French'),
 ('author', 'author', 'author', 'author'),
 ('who', 'who', 'who', 'who'),
 ('helped', 'help', 'help', 'helped'),
 ('pioner', 'pioneer', 'pioneer', 'pioneer'),
 ('the', 'the', 'the', 'the'),
 ('science-fiction', 'sciencefiction', 'sciencefict', 'sciencefiction'),
 ('genre', 'genre', 'genr', 'genre'),
 ('.', '.', '.', '.'),
 ('Verne', 'Verne', 'vern', 'Verne'),
 ('wrate', 'write', 'wrote', 'wrote'),
 ('about', 'about', 'about', 'about'),
 ('space', 'space', 'space', 'space'),
 (',', ',', ',', ','),
 ('air', 'air', 'air', 'air'),
 (',', ',', ',', ','),
 ('and', 'and', 'and', 'and'),
 ('underwater', 'underwater', 'underwat', 'underwater'),
 ('travel', 'travel', 'travel', 'travel'),
 ('before', 'before', 'befor', 'before'),
 ('navigable', 'navigable', 'navig', 'navigable'),
 ('aircrast', 'aircraft', 'aircraft', 'aircraft'),
 ('and', 'and', 'and', 'and'),
 ('practical', 'practical', 'practic', 'practical'),
 ('submarines', 'submarine', 'submarin', '

In [0]:
df = pd.DataFrame({"token":result['token'],
                   "corrected":result['spell'],
                   'POS':result['pos'],
                   'lemmas':result['lemmas'],
                   'stems':result['stems']}) 
df

,token,corrected,POS,lemmas,stems
0,French,French,JJ,French,french
1,author,author,NN,author,author
2,who,who,WP,who,who
3,helped,helped,VBD,help,help
4,pioner,pioneer,NN,pioneer,pioneer
5,the,the,DT,the,the
6,science-fiction,sciencefiction,NN,sciencefiction,sciencefict
7,genre,genre,NN,genre,genr
8,.,.,.,.,.
9,Verne,Verne,NNP,Verne,vern


## Explain Document DL

In [0]:
pipeline_dl = PretrainedPipeline('explain_document_dl', 'en')

explain_document_dl download started this may take some time.
Approx size to download 168.4 MB
[OK!]


In [0]:
%%time
result = pipeline_dl.annotate(testDoc)

CPU times: user 33.1 ms, sys: 12.7 ms, total: 45.8 ms
Wall time: 724 ms


In [0]:
result.keys()

dict_keys(['entities', 'stem', 'checked', 'lemma', 'document', 'pos', 'token', 'ner', 'embeddings', 'sentence'])

In [0]:
result['entities']

['French', 'Verne']

In [0]:
df = pd.DataFrame({"token":result['token'],
                   "ner_label":result['ner'],
                   "spell_corrected":result['checked'],
                   'POS':result['pos'],
                   'lemma':result['lemma'],
                   'stem':result['stem']}) 
df

,token,ner_label,spell_corrected,POS,lemma,stem
0,French,B-MISC,French,JJ,French,french
1,author,O,author,NN,author,author
2,who,O,who,WP,who,who
3,helped,O,helped,VBD,help,help
4,pioner,O,pioneer,NN,pioneer,pioneer
5,the,O,the,DT,the,the
6,science-fiction,O,sciencefiction,NN,sciencefiction,sciencefict
7,genre,O,genre,NN,genre,genr
8,.,O,.,.,.,.
9,Verne,B-PER,Verne,NNP,Verne,vern


## Spell Checking

In [0]:
spell_checker = PretrainedPipeline('check_spelling', 'en')

check_spelling download started this may take some time.
Approx size to download 892.6 KB
[OK!]


In [0]:
result = spell_checker.annotate(testDoc)

In [0]:
result.keys()

dict_keys(['document', 'sentence', 'token', 'checked'])

In [0]:
list(zip(result['token'], result['checked']))

[('French', 'French'),
 ('author', 'author'),
 ('who', 'who'),
 ('helped', 'helped'),
 ('pioner', 'pioneer'),
 ('the', 'the'),
 ('science-fiction', 'science-fiction'),
 ('genre', 'genre'),
 ('.', '.'),
 ('Verne', 'Vern'),
 ('wrate', 'wrote'),
 ('about', 'about'),
 ('space', 'space'),
 (',', ','),
 ('air', 'air'),
 (',', ','),
 ('and', 'and'),
 ('underwater', 'underwater'),
 ('travel', 'travel'),
 ('before', 'before'),
 ('navigable', 'navigable'),
 ('aircrast', 'aircraft'),
 ('and', 'and'),
 ('practical', 'practical'),
 ('submarines', 'submarines'),
 ('were', 'were'),
 ('invented', 'invented'),
 (',', ','),
 ('and', 'and'),
 ('before', 'before'),
 ('any', 'any'),
 ('means', 'means'),
 ('of', 'of'),
 ('space', 'space'),
 ('travel', 'travel'),
 ('had', 'had'),
 ('been', 'been'),
 ('devised', 'devised'),
 ('.', '.')]

## Parsing a list of texts

In [0]:
testDoc_list = [ 'French author who helped pioner the science-fiction genre.',
'Verne wrate about space, air, and underwater travel before navigable aircrast',
'Practical submarines were invented, and before any means of space travel had been devised.']

In [0]:
result_list = pipeline.annotate(testDoc_list)

In [0]:
len(result_list)

3

In [0]:
result_list[0]

{'document': ['French author who helped pioner the science-fiction genre.'],
 'lemmas': ['French',
  'author',
  'who',
  'help',
  'pioneer',
  'the',
  'sciencefiction',
  'genre',
  '.'],
 'pos': ['JJ', 'NN', 'WP', 'VBD', 'NN', 'DT', 'NN', 'NN', '.'],
 'sentence': ['French author who helped pioner the science-fiction genre.'],
 'spell': ['French',
  'author',
  'who',
  'helped',
  'pioneer',
  'the',
  'sciencefiction',
  'genre',
  '.'],
 'stems': ['french',
  'author',
  'who',
  'help',
  'pioneer',
  'the',
  'sciencefict',
  'genr',
  '.'],
 'token': ['French',
  'author',
  'who',
  'helped',
  'pioner',
  'the',
  'science-fiction',
  'genre',
  '.']}

## Using fullAnnotate to get more details

In [0]:
text = 'Peter Parker is a nice guy and lives in New York'

In [0]:
detailed_result = pipeline_dl.fullAnnotate(text)

In [0]:
detailed_result

[{'checked': [Annotation(token, 0, 4, Peter, {'confidence': '1.0', 'sentence': '0'}),
   Annotation(token, 6, 11, Parker, {'confidence': '1.0', 'sentence': '0'}),
   Annotation(token, 13, 14, is, {'confidence': '1.0', 'sentence': '0'}),
   Annotation(token, 16, 16, a, {'confidence': '1.0', 'sentence': '0'}),
   Annotation(token, 18, 21, nice, {'confidence': '1.0', 'sentence': '0'}),
   Annotation(token, 23, 25, guy, {'confidence': '1.0', 'sentence': '0'}),
   Annotation(token, 27, 29, and, {'confidence': '1.0', 'sentence': '0'}),
   Annotation(token, 31, 35, lives, {'confidence': '1.0', 'sentence': '0'}),
   Annotation(token, 37, 38, in, {'confidence': '1.0', 'sentence': '0'}),
   Annotation(token, 40, 42, New, {'confidence': '1.0', 'sentence': '0'}),
   Annotation(token, 44, 47, York, {'confidence': '1.0', 'sentence': '0'})],
  'document': [Annotation(document, 0, 47, Peter Parker is a nice guy and lives in New York, {})],
  'embeddings': [Annotation(word_embeddings, 0, 4, Peter, {'is

In [0]:
detailed_result[0]['entities']

[Annotation(chunk, 0, 11, Peter Parker, {'entity': 'PER', 'sentence': '0', 'chunk': '0'}),
 Annotation(chunk, 40, 47, New York, {'entity': 'LOC', 'sentence': '0', 'chunk': '1'})]

In [0]:
detailed_result[0]['entities'][0]

Annotation(chunk, 0, 11, Peter Parker, {'entity': 'PER', 'sentence': '0', 'chunk': '0'})

In [0]:
# To reach the value we use .result method
detailed_result[0]['entities'][0].result

'Peter Parker'

In [0]:
detailed_result[0]['entities'][0].metadata['entity']

'PER'

In [0]:
chunks = []
entities = []

for n in detailed_result[0]['entities']:
    chunks.append(n.result)
    entities.append(n.metadata['entity'])

df = pd.DataFrame({'chunks':chunks, 'entities':entities})

In [0]:
df

,sent_id,token,start,end,pos,ner
0,0,Peter,0,4,NNP,B-PER
1,0,Parker,6,11,NNP,I-PER
2,0,is,13,14,VBZ,O
3,0,a,16,16,DT,O
4,0,nice,18,21,JJ,O
5,0,guy,23,25,NN,O
6,0,and,27,29,CC,O
7,0,lives,31,35,NNS,O
8,0,in,37,38,IN,O
9,0,New,40,42,NNP,B-LOC


In [0]:
tuples = []

for x, y, z in zip(detailed_result[0]["token"], detailed_result[0]['pos'], detailed_result[0]['ner']):
    tuples.append((int(x.metadata['sentence']), x.result, x.begin, x.end, y.result, z.result))

df = pd.DataFrame(tuples, columns=['sent_id', 'token', 'start', 'end', 'pos', 'ner'])
df

,sent_id,token,start,end,pos,ner
0,0,Peter,0,4,NNP,B-PER
1,0,Parker,6,11,NNP,I-PER
2,0,is,13,14,VBZ,O
3,0,a,16,16,DT,O
4,0,nice,18,21,JJ,O
5,0,guy,23,25,NN,O
6,0,and,27,29,CC,O
7,0,lives,31,35,NNS,O
8,0,in,37,38,IN,O
9,0,New,40,42,NNP,B-LOC


## Use Pretrained **match_chunk** Pipeline for individual Noun Phrase

pipeline uses the regex pattern of `<DT>?<JJ>*<NN>+` (yani determinat (DT) etiher the or a might occur, one or more adjective (JJ) and noun (NN) 

In [0]:
pipeline = PretrainedPipeline('match_chunks', 'en')


match_chunks download started this may take some time.
Approx size to download 4.3 MB
[OK!]


In [0]:
result = pipeline.annotate('The book has many chapters. The red car seems awesome')

In [0]:
result

{'chunk': ['The book', 'The red car'],
 'document': ['The book has many chapters. The red car seems awesome'],
 'pos': ['DT', 'NN', 'VBZ', 'JJ', 'NNS', '.', 'DT', 'JJ', 'NN', 'VBZ', 'JJ'],
 'sentence': ['The book has many chapters.', 'The red car seems awesome'],
 'token': ['The',
  'book',
  'has',
  'many',
  'chapters',
  '.',
  'The',
  'red',
  'car',
  'seems',
  'awesome']}

In [0]:
result = pipeline.annotate('the little brown pitty bird was trying to fly and the green dog was barking at it')

In [0]:
result

{'chunk': ['the little brown pitty bird', 'the green dog'],
 'document': ['the little brown pitty bird was trying to fly and the green dog was barking at it'],
 'pos': ['DT',
  'JJ',
  'JJ',
  'NN',
  'NN',
  'VBD',
  'VBG',
  'TO',
  'VB',
  'CC',
  'DT',
  'JJ',
  'NN',
  'VBD',
  'VBG',
  'IN',
  'PRP'],
 'sentence': ['the little brown pitty bird was trying to fly and the green dog was barking at it'],
 'token': ['the',
  'little',
  'brown',
  'pitty',
  'bird',
  'was',
  'trying',
  'to',
  'fly',
  'and',
  'the',
  'green',
  'dog',
  'was',
  'barking',
  'at',
  'it']}

In [0]:
result['chunk']

['the little brown pitty bird']

## Extract Exact Dates afrom Referential Date Phrahes

In [0]:
pipeline = PretrainedPipeline('match_datetime', 'en')

match_datetime download started this may take some time.
Approx size to download 12.9 KB
[OK!]


In [0]:
result = pipeline.annotate('I saw him yesterday and he told me that he would visit us next week.')

In [0]:
result

{'date': ['2020/04/27', '2020/04/19'],
 'document': ['I saw him yesterday and he told me that he would visit us next week.'],
 'sentence': ['I saw him yesterday and he told me that he would visit us next week.'],
 'token': ['I',
  'saw',
  'him',
  'yesterday',
  'and',
  'he',
  'told',
  'me',
  'that',
  'he',
  'would',
  'visit',
  'us',
  'next',
  'week',
  '.']}

In [0]:
result = pipeline.fullAnnotate('I saw him yesterday and he told me that he would visit us next week.')

In [0]:
result

[{'date': [Annotation(date, 58, 66, 2020/04/27, {'sentence': '0'}),
   Annotation(date, 10, 18, 2020/04/19, {'sentence': '0'})],
  'document': [Annotation(document, 0, 67, I saw him yesterday and he told me that he would visit us next week., {})],
  'sentence': [Annotation(document, 0, 67, I saw him yesterday and he told me that he would visit us next week., {'sentence': '0'})],
  'token': [Annotation(token, 0, 0, I, {'sentence': '0'}),
   Annotation(token, 2, 4, saw, {'sentence': '0'}),
   Annotation(token, 6, 8, him, {'sentence': '0'}),
   Annotation(token, 10, 18, yesterday, {'sentence': '0'}),
   Annotation(token, 20, 22, and, {'sentence': '0'}),
   Annotation(token, 24, 25, he, {'sentence': '0'}),
   Annotation(token, 27, 30, told, {'sentence': '0'}),
   Annotation(token, 32, 33, me, {'sentence': '0'}),
   Annotation(token, 35, 38, that, {'sentence': '0'}),
   Annotation(token, 40, 41, he, {'sentence': '0'}),
   Annotation(token, 43, 47, would, {'sentence': '0'}),
   Annotation(to

## Sentiment Analysis

In [0]:
pipeline =PretrainedPipeline('analyze_sentiment', 'en')

analyze_sentiment download started this may take some time.
Approx size to download 4.9 MB
[OK!]


In [0]:
result = pipeline.annotate('The movie I watched was not a good one')

In [0]:
result

{'checked': ['The', 'movie', 'I', 'watched', 'was', 'not', 'a', 'good', 'one'],
 'document': ['The movie I watched was not a good one'],
 'sentence': ['The movie I watched was not a good one'],
 'sentiment': ['negative'],
 'token': ['The', 'movie', 'I', 'watched', 'was', 'not', 'a', 'good', 'one']}

In [0]:
result['sentiment']

['negative']